email agent
- authenticates user
    - only then are they allowed into the "inbox"
    - dynamic tools and prompt on the condition of there being an email and password in state that match hardcoded
- checks "inbox"
    - email in tool
- sends emails
    - human in the loop

In [1]:
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
from dataclasses import dataclass

@dataclass
class EmailContext:
    email_address: str = "julie@example.com"
    password: str = "password123"

In [3]:
from langchain.agents import AgentState

class AuthenticatedState(AgentState):
    authenticated: bool

In [4]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command(update={
            "authenticated": True, 
            "messages": [ToolMessage(
                "Successfully authenticated", 
                tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed", 
                tool_call_id=runtime.tool_call_id)]
        })

In [5]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Allow read inbox and send email tools only if user provides correct email and password"""

    authenticated = request.state.get("authenticated")
    
    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools) 
    return handler(request)

In [6]:
from langchain.agents.middleware import dynamic_prompt

authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

In [7]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

agent = create_agent(
    "gpt-5-nano",
    tools=[authenticate, check_inbox, send_email],
    checkpointer=InMemorySaver(),
    state_schema=AuthenticatedState,
    context_schema=EmailContext,
    middleware=[
        dynamic_tool_call, 
        dynamic_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "authenticate": False,
                "check_inbox": False,
                "send_email": True,
            })
        ]
    )


In [8]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="Please check my inbox. My email address is julie@example.com and my password is password123.")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

/home/codespace/.local/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=EmailContext(email_addres... password='password123'), input_type=EmailContext])
  return self.__pydantic_serializer__.to_python(


I checked your inbox. Here’s the latest message:

- From: Jane <jane@example.com>
- Message: "Hi Julie, I'm going to be in town next week and was wondering if we could grab a coffee? - best, Jane (jane@example.com)"

Would you like me to reply? I can draft a few options or tailor a message for you. Here are a couple of ready-to-send drafts:

Draft 1 (casual):
Hi Jane! That sounds great. I’d love to catch up. What day and time work for you next week?

Draft 2 (more specific):
Hi Jane, that sounds lovely. I’m free Tue or Thu afternoon next week—do either of those work for you? If you have a preferred cafe, I’m happy to meet there.

If you want, tell me:
- Your preferred day(s) and time window
- Any cafe or location you’d like to meet
- A warmer or more formal tone

I can send a reply right away once you pick a draft or provide your preferences.


In [14]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="Send an email to NameLastName@hotmail.com subject Test and This is a test as content")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

In [16]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

This is a test


In [17]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

print(response["messages"][-1].content)

Email sent to NameLastName@hotmail.com with subject Test and body This is a test


In [18]:
from pprint import pprint

pprint(response)

{'authenticated': True,
 'messages': [HumanMessage(content='Please check my inbox. My email address is julie@example.com and my password is password123.', additional_kwargs={}, response_metadata={}, id='59314e8b-9fab-472c-a90a-c5c8eda98b1e'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 222, 'prompt_tokens': 164, 'total_tokens': 386, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3RhrMkWgLB916rYpP9NbLNkKm5Wf', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c0b2a-7bcf-72d0-9424-a1f1c8f5ec0c-0', tool_calls=[{'name': 'authenticate', 'args': {'email': 'julie@example.com', 'password': 'password12